# Conway's Game of Life
using PyTorch for fun

In [5]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
%matplotlib inline

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
def play(board_size=100, epochs=10):
  kernel = torch.ones(1,1,3,3).int().to(device)
  kernel[0,0,1,1] = 0
  board = torch.randint(2,(1, 1, board_size, board_size)).int().to(device)
  frames = [board[0, 0]]
  for i in range(epochs):
    board = play_one(board, kernel)
    frames.append(board[0, 0])
  return frames

def play_one(board, kernel):
  neighbors = F.conv2d(board, kernel, padding='same')
  new_board = (neighbors == 3) | ((board == 1) & (neighbors == 2)).int()
  return new_board

def draw(frames, figsize=(6, 6),):
  def get_frame(ax):
    def f(d):
      ax.clear()
      ax.axis('off')
      ax.margins(0)
      ax.imshow(frames[d], cmap='binary')
    return f

  fig, ax = plt.subplots(figsize=figsize)
  fig.tight_layout()
  ani = FuncAnimation(fig, get_frame(ax), frames=len(frames), interval=50, repeat=False)
  plt.close()
  display(HTML(ani.to_jshtml())) # display inline
  ani.save('out/gol.gif', fps=20) # save to disk

In [10]:
frames = play(100, 100)
draw(frames)